#  2.1 Графический анализ данных

In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from collections import Counter
import numpy as np

init_notebook_mode(connected=True)

### Виды случайной величины

**Случайной величиной** называют величину, которая в результате испытания примет одно возможное значение, наперёд неизвестное и зависящее от случайных причин, которые заранее не могут быть учтены.

Выпадение некоторого значения случайной величины Х это случайное событие: Х = хi. Среди случайных величин выделяют дискретные и непрерывные случайные величины.

**Дискретной случайной величиной** называется случайная величина, которая в результате испытания принимает отдельные значения с определёнными вероятностями. Число возможных значений дискретной случайной величины может быть конечным и бесконечным. Примеры дискретной случайной величины: запись показаний спидометра или измеренной температуры в конкретные моменты времени.

**Непрерывной случайной величиной** называют случайную величину, которая в результате испытания принимает все значения из некоторого числового промежутка. Число возможных значений непрерывной случайной величины бесконечно. Пример непрерывной случайной величины: измерение скорости перемещения любого вида транспорта или температуры в течение конкретного интервала времени.

### Ряды распределений

Выделяют три формы вариационного ряда: ранжированный ряд, дискретный ряд и интервальный ряд.

**Ранжированный ряд** - это распределение отдельных единиц совокупности в порядке возрастания или убывания исследуемого признака. Ранжирование позволяет легко разделить количественные данные по группам, сразу обнаружить наименьшее и наибольшее значения признака, выделить значения, которые чаще всего повторяются.

Другие формы вариационного ряда - групповые таблицы, составленные по характеру вариации значений изучаемого признака. По характеру вариации различают дискретные (прерывные) и непрерывные признаки.

**Дискретный ряд** - это такой вариационный ряд, в основу построения которого положены признаки с прерывным изменением (дискретные признаки). К последним можно отнести тарифный разряд, количество детей в семье, число работников на предприятии и т.д. Эти признаки могут принимать только конечное число определенных значений.

Дискретный вариационный ряд представляет таблицу, которая состоит из двух граф. В первой графе указывается конкретное значение признака, а во второй - число единиц совокупности с определенным значением признака.

Если признак имеет непрерывное изменение (размер дохода, стаж работы, стоимость основных фондов предприятия и т.д., которые в определенных границах могут принимать любые значения), то для этого признака нужно строить **интервальный вариационный ряд**.

In [2]:
discrete_series = [12, 8, 5, 10, 6, 2, 6, 7, 4, 7, 10]
continuous_series = [2.1, 3.9, 5.2, 6.5, 6.3, 7.7, 7.2, 8.1, 10.1, 10.5, 12.9]
rang_discrete_series = Counter(sorted(discrete_series))
rang_discrete_series

Counter({2: 1, 4: 1, 5: 1, 6: 2, 7: 2, 8: 1, 10: 2, 12: 1})

### Гистограммы

In [3]:
iplot([go.Bar(x=list(rang_discrete_series.keys()),
              y=list(rang_discrete_series.values()))])

In [4]:
iplot([go.Histogram(x=continuous_series,
                    xbins=dict(size=3))])

### Ящик с усами

In [5]:
data_box = [go.Box(y=discrete_series,
                   boxpoints='all',
                   pointpos=-1.5,
                   name='discrete_series')]
layout_box = go.Layout(boxgap=0.8)

iplot(go.Figure(data=data_box, layout=layout_box))

In [6]:
discrete_series_with_outlier = discrete_series+[18]
data_box = [go.Box(y=discrete_series,
                   boxpoints='all',
                   pointpos=-1.5,
                   name='discrete_series'),
            go.Box(y=discrete_series_with_outlier,
                   boxpoints='all',
                   pointpos=-1.5,
                  name='discrete_series_with_outlier')]
layout_box = go.Layout(boxgap=0.7)

iplot(go.Figure(data=data_box, layout=layout_box))

### Удаление выбросов
**Межквартильный размах** - это разность между третьим и первым квартилями, является характеристикой разброса распределения.

In [7]:
Q1, Q2, Q3 = np.percentile(discrete_series_with_outlier, [25, 50, 75])
IQR = Q3 - Q1
lower_inner_fence = Q1 - (1.5 * IQR)
lower_outer_fence = Q1 - (3 * IQR)
upper_inner_fence = Q3 + (1.5 * IQR)
upper_outer_fence = Q3 + (3 * IQR)
print(f'''upper_outer_fence:  {upper_outer_fence}, 
upper_inner_fence:  {upper_inner_fence},
lower_inner_fence:  {lower_inner_fence},
lower_outer_fence"  {lower_outer_fence}''')
print('discrete_series_with_outlier   ', discrete_series_with_outlier)
print('discrete_series_without_outlier',
      list(filter(lambda x: lower_inner_fence < x < upper_inner_fence,
                  discrete_series_with_outlier)))

upper_outer_fence:  22.75, 
upper_inner_fence:  16.375,
lower_inner_fence:  -0.625,
lower_outer_fence"  -7.0
discrete_series_with_outlier    [12, 8, 5, 10, 6, 2, 6, 7, 4, 7, 10, 18]
discrete_series_without_outlier [12, 8, 5, 10, 6, 2, 6, 7, 4, 7, 10]


# 2.2 Очистка данных
### Сбор данных

In [8]:
import requests
import re
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from collections import Counter

In [9]:
url_tf_idf = 'http://nlpx.net/archives/57'
r = requests.get(url_tf_idf)
soup = BeautifulSoup(r.text, 'lxml')
text = ' '.join(map(lambda x: x.text, soup.select('.post-content')))
print('Text:', text[:300])
tokens = re.findall(r"[\w']+", text.lower())
print('\nTokens:\n\n', tokens[:50])

Text: 

Здесь я расскажу и покажу в примерах на Python, зачем и как считать стандартный TF-IDF, а также его вариации. Примеры я буду давать по ходу объяснения. Чтобы их понять, нужно иметь представления о базовых понятиях языка программирования Python в версии 2.х либо 3.х (основные типы данных, основные 

Tokens:

 ['здесь', 'я', 'расскажу', 'и', 'покажу', 'в', 'примерах', 'на', 'python', 'зачем', 'и', 'как', 'считать', 'стандартный', 'tf', 'idf', 'а', 'также', 'его', 'вариации', 'примеры', 'я', 'буду', 'давать', 'по', 'ходу', 'объяснения', 'чтобы', 'их', 'понять', 'нужно', 'иметь', 'представления', 'о', 'базовых', 'понятиях', 'языка', 'программирования', 'python', 'в', 'версии', '2', 'х', 'либо', '3', 'х', 'основные', 'типы', 'данных', 'основные']


### Удаленеи выбросов
**Стоп-слова** — слова, которые не несут никакой смысловой нагрузки. Их еще называют шумовыми словами. Например,в английском языке — это артикли, в русском — междометия, союзы, маты и т.д.

In [10]:
term_freq_dict = Counter(tokens)
term_freq_tuple = [(key, value) for key, value in term_freq_dict.items()]
sorted(term_freq_tuple, key=lambda x: x[1], reverse=True)[:200]

[('в', 43),
 ('0', 43),
 ('tf', 32),
 ('idf', 32),
 ('1', 32),
 ("'la'", 28),
 ('i', 27),
 ('text', 26),
 ('tf_text', 26),
 ('а', 24),
 ('и', 21),
 ("'vista'", 21),
 ('counter', 19),
 ("'baby'", 19),
 ('in', 18),
 ('документов', 17),
 ('количество', 17),
 ('corpus', 16),
 ('термин', 15),
 ('на', 14),
 ('можно', 14),
 ('термина', 14),
 ('для', 14),
 ('for', 13),
 ("'hasta'", 13),
 ('word', 13),
 ('слов', 12),
 ('compute_tf', 12),
 ("'siempre'", 12),
 ('это', 11),
 ('тексте', 11),
 ('import', 11),
 ('def', 11),
 ('return', 11),
 ('что', 10),
 ('встречается', 10),
 ('слова', 10),
 ('math', 10),
 ('2', 9),
 ('collections', 9),
 ('всех', 9),
 ('есть', 9),
 ('print', 9),
 ('out', 9),
 ('то', 8),
 ('или', 8),
 ('compute_idf', 8),
 ('5', 8),
 ('считать', 7),
 ('раз', 7),
 ('когда', 7),
 ('с', 7),
 ('логарифм', 7),
 ('texts', 7),
 ('la', 7),
 ('я', 6),
 ('как', 6),
 ('frequency', 6),
 ('встретился', 6),
 ('виде', 6),
 ("'pasta'", 6),
 ("'comandante'", 6),
 ('baby', 6),
 ('siempre', 6),
 ('часто

In [11]:
Q1, Q2, Q3 = np.percentile(list(term_freq_dict.values()), [25, 50, 75])
IQR = Q3 - Q1
lower_inner_fence = Q1 - (1.5 * IQR)
lower_outer_fence = Q1 - (3 * IQR)
upper_inner_fence = Q3 + (1.5 * IQR)
upper_outer_fence = Q3 + (3 * IQR)
print(f'''upper_outer_fence:  {upper_outer_fence}, 
upper_inner_fence:  {upper_inner_fence},
lower_inner_fence:  {lower_inner_fence},
lower_outer_fence"  {lower_outer_fence}''')

upper_outer_fence:  5.0, 
upper_inner_fence:  3.5,
lower_inner_fence:  -0.5,
lower_outer_fence"  -2.0


### Дополнительно
***N*-граммы** — последовательность из *n* элементов. С семантической точки зрения, это может быть последовательность звуков, слогов, слов или букв. На практике чаще встречается *N*-грамма как ряд слов, устойчивые словосочетания называют коллокацией. Последовательность из двух последовательных элементов часто называют биграмма, последовательность из трёх элементов называется триграмма.  
**TF-IDF** — статистическая мера, используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса. Вес некоторого слова пропорционален частоте употребления этого слова в документе и обратно пропорционален частоте употребления слова во всех документах коллекции.